## Road Graph Creation Workflow
This notebook creates a stored road graph file so that the Osmnx API is not called for every run and the same cleaning techniques are used for each road graph. This saves time that would otherwise be needed to rebuild graphs for each simulation run.

In [ ]:
# import functions from main code base
from FireABM_opt import *

In [ ]:
# set road graph parameters - these were used in the accompanying manuscript
central_addresses = ['100 Santa Rosa Ave, Santa Rosa, CA 95404']
names = ['Sta_Rosa']
hhs = ["Santa_Rosa_tracts_hh.shp"]
fires = ["santa_rosa_fire.shp"]
distances = [2000, 8000]

In [ ]:
for indx in range(len(central_addresses)):
    for dist in distances:
        road_graph = ox.graph_from_address(
            central_addresses[indx], distance=dist, distance_type='bbox', network_type='drive', 
            clean_periphery=True, truncate_by_edge=True)
        
        road_graph = ox.add_edge_bearings(road_graph)
        road_graph = ox.project_graph(road_graph) # projects graph
        road_graph = cleanUp(road_graph) # fills missing speeds, removes dead ends
        
        road_graph = add_unit_speed(road_graph) # adds unit speed
        road_graph = add_road_type_weights(road_graph) # adds weights for road types
        
        gdf_nodes, gdf_edges = get_node_edge_gdf(road_graph)
        (bbox, lbbox, poly, x, y) = create_bboxes(gdf_nodes, 0.01)
        
        hh_tract = load_shpfile(road_graph, ("households", hhs[indx]))
        road_graph, overlap = add_households(road_graph, hh_tract, hhs[indx], 
                                             "Tot_Est_HH", num_col=4, bbox_poly=poly) # adds households
        
        fire_file = load_shpfile(road_graph, ("fire_input",fires[indx]))
        init_fire = fire_file[fire_file['SimTime'] == 60]
        road_graph = add_fire_distance(road_graph, init_fire) # adds distance from fire
        
        gdf_nodes, gdf_edges = get_node_edge_gdf(road_graph)
        
        # normalizes some attributes
        road_graph = normalize_edge_attribute(road_graph, 'length', min(gdf_edges.length), 
                                      max(gdf_edges.length), 'length_n')
        road_graph = normalize_edge_attribute(road_graph, 'rt_weighted_len', min(gdf_edges.rt_weighted_len), 
                                      max(gdf_edges.rt_weighted_len), 'rt_wght_len_n')
        road_graph = normalize_edge_attribute(road_graph, 'fire_dist', min(gdf_edges.fire_dist), 
                                      max(gdf_edges.fire_dist), 'fire_dist_n')
        road_graph = invert_norm_edge_attribute(road_graph, 'fire_dist_n', 'inv_fire_dist_n')
        
        outname = names[indx]+'_'+str(dist)+'.pkl'
        nx.write_gpickle(road_graph, path=outname)
        print(outname)